In [8]:
# 设置环境变量，配置 API 地址与密钥（使用第三方 OpenAI 镜像）
import os
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech"
os.environ["OPENAI_API_KEY"] = 'sk-xxx'

In [9]:
# -------------------- 文档加载与切分 --------------------
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader

base_dir = 'Docs'  # 文档所在文件夹路径
documents = []     # 存放加载后的 Document 对象

# 遍历文件夹下所有文件，根据文件类型调用不同的加载器
for file in os.listdir(base_dir):
    file_path = os.path.join(base_dir, file)
    if file.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())
    elif file.endswith('.docx'):
        loader = Docx2txtLoader(file_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        loader = TextLoader(file_path)
        documents.extend(loader.load())

# 文本切分器：将长文档切分为多个段落块，chunk_size 为每段最大长度
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)

In [10]:
# -------------------- 嵌入 + 存储到向量数据库 --------------------
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

# 将切分后的文档嵌入为向量，并临时存储在 Qdrant 的内存数据库中
vectorstore = Qdrant.from_documents(
    documents=chunked_documents,
    embedding=OpenAIEmbeddings(),
    location=":memory:",  # 仅存于内存，适合教学/测试
    collection_name="my_documents"
)

In [11]:
# 准备模型和Retrieval链
import logging # 导入Logging工具
from langchain_openai import ChatOpenAI # ChatOpenAI模型
from langchain.retrievers.multi_query import MultiQueryRetriever # MultiQueryRetriever工具
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


# 设置Logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

# 实例化一个大模型工具 - OpenAI的GPT-3.5
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 实例化一个MultiQueryRetriever
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(), llm=llm)

# 实例化一个memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.clear()

# 实例化一个ConversationalRetrievalChain
qa_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever_from_llm, memory=memory, verbose=True)

INFO:langchain.retrievers.multi_query:Generated queries: ['MCP的定义是什么？  ', 'MCP指的是什么？  ', 'MCP的全称和解释是什么？']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
MCP调研报告

1. 什么是MCP（Model Context Protocol）

MCP（Model Context Protocol，模型上下文协议） ，2024年11月底，由 Anthropic 推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。MCP 的主要目的在于解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的难题，MCP 使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供了连接万物的接口。

MCP 客户端（MCP Clients）：由客户端LLM应用使用Client SDK创建并维护的一个Server会话。

MCP 服务器（MCP Servers）：为Client  提供上下文、工具和 prompt 等信息的服务。可以部署在LLM应用本机，也可以远程部署（Remote）。



3.2 工作流程：

[该类型的内容暂不支持下载]

MCP 服务器（MCP Servers）

这也就意味着，如果我们要发开一个 Function Call 工具，需要对不同的模型进行适配，比如参数格式、触发逻辑、返回结构等等，这个成本是非常高的。

阶段三、MCP（ Anthropic  在 2024 .11 提出）















MCP

核心贡献：统一规范

Command 对应命令，args 对应参数。对于公开的 MCP 工具，我们通常可以这样找到相应的设置方法，后面我还会介绍如何查找更多公开的 MCP 服务器。

4.2.4 使用方法

安装完服务器后，在对话界面点击启用 MCP 服务器。

选择支持 MCP 的模

INFO:langchain.retrievers.multi_query:Generated queries: ['它是如何使用的？', '请问这个产品的具体使用步骤是什么？', '如何正确使用它？']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
做什么。老人笑着回答说，他在做一只小熊玩具，准备送给自己的孙子。\n\n小女孩感动地看着',

哈哈，看来小明的“数学”逻辑真是与众不同呀！
Predict method took 2.9873 seconds to execute.
有一天，小明去上学，老师问他：“小明，如果你有十个苹果，给了别人四个，你还剩几个苹果？”  
小明想了想，回答：“老师，我还剩下六个！”  
老师说：“不对啊，应该是六个。你为什么会那么确定？”

代码调用

智能体平台（如dify、阿里百炼）

4.2 使用示例

以CherryStudio为例子

4.2.1 为什么选择 Cherry Studio？

Command 对应命令，args 对应参数。对于公开的 MCP 工具，我们通常可以这样找到相应的设置方法，后面我还会介绍如何查找更多公开的 MCP 服务器。

4.2.4 使用方法

安装完服务器后，在对话界面点击启用 MCP 服务器。

选择支持 MCP 的模型（带有扳手 🔧 标志）。

就可以开始对话了！



使用mcp功能



选择支持mcp的模型

对话效果：



对话效果

> Finished chain.
We were just exchanging greetings and asking each other how we are. Is there something specific you would like to talk about?

集成外部工具
Human: 它的使用方法是什么？

> Finished chain.

> Finished chain.
我不知道。

In [12]:
# -------------------- 文档摘要链（Map-Reduce） --------------------
from langchain.chains.summarize import load_summarize_chain

summ_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=True)

def qa_with_summary(question: str):
    docs = retriever_from_llm.get_relevant_documents(question)
    summary = summ_chain.run(docs)
    return summary

print(qa_with_summary("请用 100 字总结 MCP 的核心概念"))

INFO:langchain.retrievers.multi_query:Generated queries: ['请用 100 字描述 MCP 的主要思想和原则。  ', 'MCP 的核心概念是什么？请用 100 字概括一下。  ', '能否用 100 字总结 MCP 的基本概念和关键点？']




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"MCP（Model Context Protocol，模型上下文协议） ，2024年11月底，由 Anthropic 推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。MCP 的主要目的在于解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的难题，MCP 使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供了连接万物的接口。"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"MCP调研报告

1. 什么是MCP（Model Context Protocol）"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"MCP 客户端（MCP Clients）：由客户端LLM应用使用Client SDK创建并维护的一个Server会话。

MCP 服务器（MCP Servers）：为Client  提供上下文、工具和 prompt 等信息的服务。可以部署在LLM应用本机，也可以远程部署（Remote）。



3.2 工作流程：

[该类型的内容暂不支持下载]

MCP 服务器（MCP Servers）"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"提供哪些服务：

Tools：提供给LLM应用特别是Agent使用的工具。

Resoures：提供给LLM应用一些额外的结构化数据。

Prompts：提供给LLM应用的一些Prompt模板。比如你的应用是一个Chatbot，可以从MCP Server中取出这些模板，让使用者选择使用。


/opt/anaconda3/envs/zjh/lib/python3.10/site-packages/langchain_openai/chat_models/base.py:386: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"MCP (Model Context Protocol), launched by Anthropic in late November 2024, is an open standard designed to unify communication between large language models (LLMs) and external data sources and tools. Its primary goal is to address data silo issues that limit AI models' potential, enabling secure access and operation of local and remote data, thereby providing a comprehensive connectivity interface for AI applications.

The MCP (Model Context Protocol) research report introduces the concept of MCP, outlining its purpose and significance in managing and interpreting contextual information within models.

MCP Clients are server sessions created and maintained by client LLM applications using a Client SDK. MCP Servers provide context, tools, and prompts to clients and can be deployed either locally or remotely.

The MCP Server offers tools for LLM applications, parti

In [13]:
# -------------------- Agent 多工具组合 --------------------
from langchain.tools import Tool
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import initialize_agent, AgentType

# 定义一个函数，用于通过Retrieval QA链回答用户的问题
def retriever_tool_func(q):
    return qa_chain({"question": q})["answer"]  # 调用qa_chain处理问题，并提取答案部分

# 创建一个用于文档检索问答的工具
retrieval_tool = Tool(name="RAG_QA", func=retriever_tool_func, description="对一般知识类问题，先检索文档再回答")

# 创建一个用于文档摘要的工具
summary_tool = Tool(name="RAG_Summary", func=qa_with_summary, description="当用户要求概括/总结时使用")

# 创建一个用于执行Python代码的工具
python_tool = PythonREPLTool()

# 将上述工具组合成一个工具列表，供Agent使用
tools = [retrieval_tool, summary_tool, python_tool]

# 初始化一个智能体（Agent），配置如下：
# - 使用OpenAI的GPT-3.5模型（llm）
# - 提供的工具列表（tools）
# - 使用ZERO_SHOT_REACT_DESCRIPTION类型的Agent（基于ReAct框架）
# - 启用详细日志输出（verbose=True）
# - 使用会话记忆（memory）以保持上下文
agent = initialize_agent(
    tools=tools,  # 提供的工具列表
    llm=llm,  # 使用的语言模型
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent的类型
    verbose=True,  # 是否输出详细的日志信息
    memory=memory  # 会话记忆，用于保持对话的上下文
)

# 示例：触发总结工具
print(agent.run("MCP 的作用能一步总结给我吗？"))

# 示例：触发 Python 计算工具
print(agent.run("计算 (27*13)+9 等于多少？"))

# 示例：触发文档检索问答工具
print(agent.run("MCP 和 LangChain Output Parser 有何区别？"))



> Entering new AgentExecutor chain...
我需要总结 MCP（可能指代“主控处理器”或其他含义）的作用。为此，我将检索相关文档来获取 MCP 的功能和作用方面的信息。  
Action: RAG_Summary  
Action Input: "MCP 的作用"  

INFO:langchain.retrievers.multi_query:Generated queries: ['MCP 在什么领域中发挥作用？  ', 'MCP 是什么，其主要功能是什么？  ', 'MCP 的具体用途和应用示例有哪些？']




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"MCP调研报告

1. 什么是MCP（Model Context Protocol）"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"MCP（Model Context Protocol，模型上下文协议） ，2024年11月底，由 Anthropic 推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。MCP 的主要目的在于解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的难题，MCP 使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供了连接万物的接口。"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"MCP 客户端（MCP Clients）：由客户端LLM应用使用Client SDK创建并维护的一个Server会话。

MCP 服务器（MCP Servers）：为Client  提供上下文、工具和 prompt 等信息的服务。可以部署在LLM应用本机，也可以远程部署（Remote）。



3.2 工作流程：

[该类型的内容暂不支持下载]

MCP 服务器（MCP Servers）"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"提供哪些服务：

Tools：提供给LLM应用特别是Agent使用的工具。

Resoures：提供给LLM应用一些额外的结构化数据。

Prompts：提供给LLM应用的一些Prompt模板。比如你的应用是一个Chatbot，可以从MCP Server中取出这些模板，让使用者选择使用。


/opt/anaconda3/envs/zjh/lib/python3.10/site-packages/langchain_openai/chat_models/base.py:386: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"The MCP (Model Context Protocol) is a framework designed to standardize the way models are contextualized and managed within various systems. It facilitates effective communication and interaction between different models by defining protocols for context sharing and management.

MCP (Model Context Protocol) is an open standard introduced by Anthropic in late November 2024, designed to unify communication between large language models (LLMs) and external data sources and tools. Its primary goal is to overcome data silos that limit AI model potential, enabling secure access and operation of local and remote data, thereby providing a comprehensive interface for AI applications.

MCP Clients are server sessions created and maintained by client LLM applications using a Client SDK. MCP Servers provide essential context, tools, and prompts to these clients, which can be

INFO:langchain.retrievers.multi_query:Generated queries: ['1. MCP 模型配置协议与 LangChain 输出解析器之间存在哪些主要区别？', '2. 在 MCP 模型配置协议和 LangChain 输出解析器的功能上，有哪些不同之处？', '3. 可以详细说明一下 MCP 模型配置协议及 LangChain 输出解析器的不同之处吗？']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
MCP调研报告

1. 什么是MCP（Model Context Protocol）

Model返回的内容通常都是字符串的模式，但在实际开发过程中，往往希望model可以返回更直观的内容，Langchain提供的输出解析器则将派上用场。

在实现一个输出解析器的过程中，需要实现两种方法：

获取格式指令：返回一个字符串的方法，其中包含有关如何格式化语言模型输出的说明。

Parse：一种接收字符串（假设是来自语言模型的响应）并将其解析为某种结构的方法。

MCP（Model Context Protocol，模型上下文协议） ，2024年11月底，由 Anthropic 推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。MCP 的主要目的在于解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的难题，MCP 使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供了连接万物的接口。

新的AI能力共享生态：通过MCP Server的共享，新的LLM应用可以快速获得各种工具，形成了一种新的合作体系，提高整体效用。

总结：FunctionCall和MCP的区别



3. 核心架构

3.1 概念解析：



MCP 主机（MCP Hosts）：发起请求的 LLM 应用程序（例如 Claude Desktop、IDE 或 AI 工具）。

这也就意味着，如果我们要发开一个 Function Call 工具，需要对不同的模型进行适配，比如参数格式、触发逻辑、返回结构等等，这个成本是非常高的。

阶段三、MCP（ Anthropic  在 2024 .

In [14]:
# -------------------- 回调打印输出事件（可用于调试、打字机效果） --------------------
from langchain_core.callbacks import BaseCallbackHandler

class SimpleCallbackHandler(BaseCallbackHandler):
    def on_llm_start(self, serialized, prompts, **kwargs):
        print("\n🔵 LLM started")

    def on_llm_new_token(self, token, **kwargs):
        print(token, end="", flush=True)

    def on_llm_end(self, response, **kwargs):
        print("\n🟢 LLM finished")

    def on_chain_end(self, outputs, **kwargs):
        print(f"\n🟡 Chain outputs: {list(outputs.keys())}")

callback_handler = SimpleCallbackHandler()

# Agent 执行带回调（打字机效果）
agent.run("LangChain 的作用能一步总结给我吗？", callbacks=[callback_handler])



> Entering new AgentExecutor chain...

🔵 LLM started



🟢 LLM finished

🟡 Chain outputs: ['text']
LangChain 是一个用于构建与大型语言模型（LLMs）交互应用程序的框架。它的主要作用是使开发者能够更容易地创建文本生成、对话代理、总结和问答等功能，通过将不同的数据源和处理模块结合起来，帮助开发者管理复杂的应用逻辑。

Action: RAG_Summary  
Action Input: "LangChain 的作用能一步总结给我吗？"  
🔵 LLM started


INFO:langchain.retrievers.multi_query:Generated queries: ['LangChain 的主要功能是什么？请简要概述一下。  ', '能否帮我概括一下 LangChain 的应用和目的？  ', '请简述一下 LangChain 的关键功能是什么？']



🟢 LLM finished


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"LangChain 系列教程（一）：入门

介绍

LangChain 是一个用于开发由语言模型驱动的应用程序的框架。它使得应用程序能够：

具有上下文感知能力：将语言模型连接到上下文来源（提示指令，少量的示例，需要回应的内容等）

具有推理能力：依赖语言模型进行推理（根据提供的上下文如何回答，采取什么行动等）

这个框架由几个部分组成。"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"LangChain 库：Python 和 JavaScript 库。包含了各种组件的接口和集成，一个基本的运行时，用于将这些组件组合成链和代理，以及现成的链和代理的实现。

LangChain 模板：一系列易于部署的参考架构，用于各种任务。

LangServe：一个用于将 LangChain 链部署为 REST API 的库。"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"LangChain 系列教程（二）：Model I/O

介绍

LangChain 提供的 LangChain Expression Language(LCEL) 让开发可以很方便地将多个组件连接成 AI 工作流。如下是一个简单的工作流："


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"LangSmith：一个开发者平台，让你可以调试、测试、评估和监控基于任何 LLM 框架构建的链，并且与 LangChain 无缝集成。 

核心模块

LangChain由6个module组成，分别是

/opt/anaconda3/envs/zjh/lib/python3.10/site-packages/langchain_openai/chat_models/base.py:386: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")



🟢 LLM finished

🟢 LLM finished

🟢 LLM finished

🟢 LLM finished

🟡 Chain outputs: ['outputs']

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"LangChain is a framework for developing applications driven by language models, enabling context-aware capabilities and reasoning. It integrates language models with contextual sources and facilitates decision-making based on provided input. The framework consists of several components.

LangChain is a library in Python and JavaScript that provides interfaces and integrations for various components, enabling users to create chains and agents. It includes ready-made implementations and templates for easy deployment across different tasks. LangServe is an additional library that allows users to deploy LangChain chains as REST APIs.

The second tutorial in the LangChain series, titled "Model I/O," introduces the LangChain Expression Language (LCEL), which enables developers 

'LangChain 是一个用于开发与大型语言模型（LLMs）互动的框架，支持创建上下文感知能力和推理的应用程序。它为 Python 和 JavaScript 提供接口，允许用户创建链和代理，并具备便于部署的模板。LangServe 可以将链部署为 REST API。LangChain 的架构包含六个核心模块：模型输入输出、检索、链、内存、代理和回调，其中心模块是模型输入输出，支持 AI 功能的实现。'

In [15]:
callback_handler = SimpleCallbackHandler()

# callbacks 可以同时传入多个 handler 的列表
agent.run("LangChain 的作用能一步总结给我吗？", callbacks=[callback_handler])



> Entering new AgentExecutor chain...

🔵 LLM started



🟢 LLM finished
🟡 Chain outputs: ['text']
LangChain 是一个用于构建基于语言模型的应用程序的框架。它提供了一系列工具和组件，帮助开发者更容易地集成和使用语言模型，支持多种功能，如文本生成、问答、对话系统等。为了更好地总结，我将使用 RAG_Summary 工具来获取更详细的信息。

Action: RAG_Summary
Action Input: "LangChain 的作用"
🔵 LLM started


INFO:langchain.retrievers.multi_query:Generated queries: ['LangChain 的功能是什么？  ', 'LangChain 可以做些什么？  ', 'LangChain 的主要用途有哪些？']



🟢 LLM finished


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"LangChain 系列教程（一）：入门

介绍

LangChain 是一个用于开发由语言模型驱动的应用程序的框架。它使得应用程序能够：

具有上下文感知能力：将语言模型连接到上下文来源（提示指令，少量的示例，需要回应的内容等）

具有推理能力：依赖语言模型进行推理（根据提供的上下文如何回答，采取什么行动等）

这个框架由几个部分组成。"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"LangChain 库：Python 和 JavaScript 库。包含了各种组件的接口和集成，一个基本的运行时，用于将这些组件组合成链和代理，以及现成的链和代理的实现。

LangChain 模板：一系列易于部署的参考架构，用于各种任务。

LangServe：一个用于将 LangChain 链部署为 REST API 的库。"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"LangChain 系列教程（二）：Model I/O

介绍

LangChain 提供的 LangChain Expression Language(LCEL) 让开发可以很方便地将多个组件连接成 AI 工作流。如下是一个简单的工作流："


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"LangSmith：一个开发者平台，让你可以调试、测试、评估和监控基于任何 LLM 框架构建的链，并且与 LangChain 无缝集成。 

核心模块

LangChain由6个module组成，分别是

/opt/anaconda3/envs/zjh/lib/python3.10/site-packages/langchain_openai/chat_models/base.py:386: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")



🟢 LLM finished

🟢 LLM finished

🟢 LLM finished

🟢 LLM finished

🟢 LLM finished
🟡 Chain outputs: ['outputs']

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"LangChain is a framework designed for developing applications powered by language models. It enables applications to have context awareness by connecting language models to various context sources and to perform reasoning based on the provided context. The framework consists of several components.

LangChain is a library for Python and JavaScript that provides interfaces and integrations for various components, a runtime for combining them into chains and agents, and ready-made implementations. It includes templates for easily deployable reference architectures for different tasks and LangServe, a library for deploying LangChain chains as REST APIs.

The second tutorial in the LangChain series, titled "Model I/O," introduces the LangChain Expression Languag

'LangChain 是一个用于开发基于语言模型的应用程序的框架，提供了多种组件和工具，支持上下文感知和推理。它包括六个核心模块：模型输入输出、检索、链、记忆、代理和回调，帮助开发者集成和部署 AI 工作流，并通过 LangServe 提供 REST API。'